<a href="https://colab.research.google.com/github/Bandaruharshini/Codsoft/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Example user-item rating matrix
data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'User5'],
    'Item1': [5, 4, 1, 0, 0],
    'Item2': [4, 0, 0, 2, 0],
    'Item3': [1, 1, 0, 5, 4],
    'Item4': [0, 1, 0, 4, 0],
    'Item5': [0, 0, 0, 3, 5]
}

# Create a DataFrame
df = pd.DataFrame(data)
df.set_index('User', inplace=True)

# Fill missing ratings (0 means not rated)
user_item_matrix = df.fillna(0)

# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Convert the similarity matrix into a DataFrame for better visualization
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("User Similarity Matrix:")
print(user_similarity_df)

# Function to recommend items to a given user based on similar users
def recommend_items(user, user_item_matrix, user_similarity_df, num_recommendations=2):
    if user not in user_item_matrix.index:
        print("User not found!")
        return []

    # Find similar users
    similar_users = user_similarity_df[user].sort_values(ascending=False)

    # Get items rated by similar users
    weighted_sum = np.zeros(user_item_matrix.shape[1])
    similarity_sum = np.zeros(user_item_matrix.shape[1])

    for sim_user in similar_users.index:
        if sim_user != user:
            similarity_score = similar_users[sim_user]
            rated_items = user_item_matrix.loc[sim_user] > 0

            weighted_sum += similarity_score * user_item_matrix.loc[sim_user].values
            similarity_sum += similarity_score * rated_items

    # Avoid division by zero
    similarity_sum[similarity_sum == 0] = 1

    # Predicted ratings
    predicted_ratings = weighted_sum / similarity_sum

    # Get the items the user hasn't rated yet
    unrated_items = user_item_matrix.loc[user] == 0

    # Recommend items with the highest predicted ratings
    recommendations = np.argsort(predicted_ratings * unrated_items.values)[::-1]
    recommended_items = [user_item_matrix.columns[i] for i in recommendations[:num_recommendations]]

    return recommended_items

# Recommend items for User1
user_to_recommend = 'User1'
recommended_items = recommend_items(user_to_recommend, user_item_matrix, user_similarity_df)
print(f"\nRecommended items for {user_to_recommend}: {recommended_items}")


User Similarity Matrix:
User      User1     User2     User3     User4     User5
User                                                   
User1  1.000000  0.763763  0.771517  0.272974  0.096393
User2  0.763763  1.000000  0.942809  0.288675  0.147242
User3  0.771517  0.942809  1.000000  0.000000  0.000000
User4  0.272974  0.288675  0.000000  1.000000  0.743839
User5  0.096393  0.147242  0.000000  0.743839  1.000000

Recommended items for User1: ['Item5', 'Item4']
